In [ ]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

def text_preprocess(mess):

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    nopunc = nopunc.lower()

    # Now just remove any stopwords and non alphabets
    nostop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english') and word.isalpha()]
    return nostop

messages = pd.read_csv("super23.csv", encoding='latin-1')
# messages = pd.read_excel('punny.xlsx')
# messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "message"]
# print (messages.head(3))
messages_train = messages.loc[37816:65736, :]
print (messages_train.tail(3))

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
messages_train["message"] = messages_train["message"].apply(text_preprocess)
messages_train["message"] = messages_train["message"].agg(lambda x: ' '.join(map(str, x)))

from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(encoding = "latin-1", strip_accents = "unicode", stop_words = "english", ngram_range=(3, 3))
# vec = TfidfVectorizer(encoding = "latin-1", strip_accents = "unicode", stop_words = "english")
features = vec.fit_transform(messages_train["message"])

In [ ]:
from sklearn.svm import SVC
clf = SVC()

# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression()

spam_detect_model = clf.fit(features, messages_train["label"])
predict_train = spam_detect_model.predict(features)
print("Classification Report \n",metrics.classification_report(messages_train["label"], predict_train))
print("\n")
print("Confusion Matrix \n",metrics.confusion_matrix(messages_train["label"], predict_train))
print("\n")
print("Accuracy of Train dataset : {0:0.3f}".format(metrics.accuracy_score(messages_train["label"], predict_train)))

In [ ]:
messages_test = messages.loc[0:37815, :]
print ("Start", messages_test.head(3))
print ("End", messages_test.tail(3))

In [ ]:
messages_test["message"] = messages_test["message"].apply(text_preprocess)
messages_test["message"] = messages_test["message"].agg(lambda x: ' '.join(map(str, x)))

features_test = vec.transform(messages_test["message"])
print(features_test.shape)

label_predictions = spam_detect_model.predict(features_test)
print(label_predictions)

print(metrics.classification_report(messages_test["label"], label_predictions))
print(metrics.confusion_matrix(messages_test["label"], label_predictions))
print("precision: ", metrics.precision_score(messages_test["label"], label_predictions))
print("recall: ", metrics.recall_score(messages_test["label"], label_predictions))
# Printing the Overall Accuracy of the model
print("Accuracy of the model: {0:0.3f}".format(metrics.accuracy_score(messages_test["label"], label_predictions)))